### Custom variable pattern

In [1]:
import gsconfig

# Словарь с переменными для замены
data = {
    'check': True,
    'nope': False,
    'variable': 'Woohooo!',
    'substring': 'subSTRing',
    'cow': 10.9,
    'int': 10,
    'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]},
    'empty': '',
    'list_in_list': [['Zero', 'One', 'Two', 'Three', 'Four']],
    'list': ['Zero', 'One', 'Two', 'Three', 'Four'],
    'list_of_int': [0, 1, 2, 3, 4]
}

# Шаблон задан как строка
template_body = """
- {% substring %} внутри строки
- Строка подстрока в кавычках: "{% variable %}"
- Явное указание на строку: {% substring!string %}
- Перевод в инты: {% cow!int %} (int) или {% cow %} (source) 

- Вот int который как float {% int!float %}
{# Я комментарий внутри шаблона #}
{% if check %}
- Тут проверка включать строку или нет ({% check %})
{% endif %}
- Словарик: {% dict %}
{% if nope %}
- А эту строку мы не включаем в результат!
{% endif %}
- И пустота: "{% empty %}"
- Беру значение из списка по индексу и обьявляю его строкой: {% list!get_2!string %}
{# Вообще команд может быть любое количество если это имеет смысл #}
- Разворачивая список из лишнего списка: {% list_in_list!extract %}
{# Тут главное оставлять комментарий по структуре данных, потом это будет невозможно вспомнить #}
- А еще можно так извлекать элементы: {% list_in_list!get_0!get_1!string %}
- Тут список не извлекаю из лишнего списка: {% list_in_list %}

{% comment %}
- Многострочные комментарии можно так делать
- Да, они правда многострочные!
{% endcomment %}
- Строка после многострочного комментария
{% foreach list %}
- Считаем в массиве: {% $item!string %}
{% endforeach %}
"""

# Класс шаблона c указанием тела шаблона и паттерна
template = gsconfig.Template(body=template_body)

# Собрать результат из шаблона и данных
r = template.render(data)
print(r)



- subSTRing внутри строки
- Строка подстрока в кавычках: "Woohooo!"
- Явное указание на строку: "subSTRing"
- Перевод в инты: 10 (int) или 10.9 (source) 

- Вот int который как float 10.0
- Тут проверка включать строку или нет (true)
- Словарик: {"key": "value", "list": [{"one": "odin"}, "two", ["three", "four", "five"]]}
- И пустота: ""
- Беру значение из списка по индексу и обьявляю его строкой: "Two"
- Разворачивая список из лишнего списка: ["Zero", "One", "Two", "Three", "Four"]
- А еще можно так извлекать элементы: "One"
- Тут список не извлекаю из лишнего списка: [["Zero", "One", "Two", "Three", "Four"]]

- Строка после многострочного комментария
- Считаем в массиве: "Zero"
- Считаем в массиве: "One"
- Считаем в массиве: "Two"
- Считаем в массиве: "Three"
- Считаем в массиве: "Four"



### Default variable pattern + template file

In [2]:
import gsconfig
import json

template_path = 'template.tpl'
template = gsconfig.Template(path=template_path, jsonify=True)  # jsonify - пытается перевести результат в JSON

# Словарь с переменными для замены
data = {
    'variable': 'Woohooo!',
    'substring': 'subSTRing',
    'cow': 10.9,
    'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]},
    'empty': ''
}

# r = json.loads(template.make(data))
r = template.render(data)
print(json.dumps(r, indent=4, ensure_ascii=False))


{
    "string": "Woohooo!",
    "string_with_command": "Woohooo!",
    "string_as_substring": "Звучит как subSTRing. Похоже?",
    "int_as_substring": "Звучит как 10.9?",
    "digit_int": 10,
    "digit_source": 10.9,
    "dict": {
        "key": "value",
        "list": [
            {
                "one": "odin"
            },
            "two",
            [
                "three",
                "four",
                "five"
            ]
        ]
    },
    "empty": ""
}


### Дополнительные команды доступные в шаблоне

In [ ]:
"""
КОМАНДЫ В КЛЮЧАХ ШАБЛОНА:
dummy -- Пустышка, ничего не длает.

float -- Переводит в начения с плавающей запятой.
Пример: Получено число 10, в шаблон оно будет записано как 10.0

int -- Переводит в целые значения отбрасыванием дробной части
Пример: Получено число 10.9, в шаблон оно будет записано как 10

json -- Сохраняет структура как JSON (применяет json.dumps())

extract -- Вытаскивает элемент из списка (list or tuple) если это список единичной длины.
Пример: По умолчанию парсер не разворачивает словари и они приходят вида [{'a': 1, 'b': 2}],
если обязательно нужен словарь, то extract развернёт полученный список до {'a': 1, 'b': 2}

wrap -- Дополнительно заворачивает полученый список если первый элемент этого списка не является списком.
Пример: Получен список [1, 2, 4], 1 - первый элемент, не список, тогда он дополнительно будет завернут [[1, 2, 4]].

string -- Дополнительно заворачивает строку в кавычки. Все прочие типы данных оставляет как есть. 
Используется когда заранее неизвестно будет ли там значение и выбор между null и строкой.
Например, в новостях мультиивентов поле "sns": {news_sns!string}.
Пример: Получена строка 'one,two,three', тогда она будет завернута в кавычки и станет '"one,two,three"'.

get_N -- Берет элемент с индексом N из списка.
Пример: Получени список (допустимы только списки и кортежи) ['Zero', 'One', 'Two'] и команда get_2
Тогда будет подставлено значение  'Two'

КОМАНДЫ УПРАВЛЕНИЯ СТРОКАМИ ШАБЛОНА:
if -- сохранить строку между тегами условия если параметр True, иначе удалить
Например, для `{% if params %} _KEEP_ME_OR_DELETE_ME_ {% endif %}`
    if -- ключевое слово условия, открывающий тег
    params -- параметр
    endif -- закрывающий тег
    строка "_KEEP_ME_OR_DELETE_ME_" будет сохранена если params == True

comment -- многострочный комментарий. Работает как if с параметром False

foreach -- экспериментальная реализация цикла. Пробегает по каждому элементу из params
Использует зарезервированную переменную $item для обозначения элемента списка. Поддерживает команды ключей.
Например, {% foreach list %}{% $item!string %}, {% endforeach %} 
для list = ('one', 'two', 'three') сделает последовательность '"one", "two", "three", '

КОММЕНТАРИИ:
{# Я комментарий в теле шаблона #}
"""